In [0]:
ls

google colab   
resnet50 在cifar-10上的准确率    
zhang yiming    
yimingzhang@kaikeba.com

In [0]:
import torch
import torchvision
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms

# 预处理的设置
# 图片转化为 alexnet 规定的图片大小
# 归一化是减去均值，除以方差
# 把 numpy array 转化为 tensor 的格式
my_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.247, 0.243, 0.261])])

# 读取数据集cifar-10的图，有10个类，5万张图片。进行预处理。
train_dataset = torchvision.datasets.CIFAR10(root='./',train=True,transform=my_tf,download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./',train=False,transform=my_tf,download=True)


In [0]:
# 调用预训练模型resnet50
my_resnet50 = torchvision.models.resnet50(pretrained=True)

In [0]:
torchvision.models.resnet50()

In [0]:
my_resnet50.fc

In [0]:
# 固定网络框架全连接层之前的参数
for param in my_resnet50.parameters():
    param.requires_grad=False

In [0]:

# 将resnet50最后一层输出的类别数，改为cifar-10的类别数（10）
in_f = my_resnet50.fc.in_features
my_resnet50.fc = nn.Linear(in_f,10)
              

In [0]:
my_resnet50

In [0]:
# 超参数设置
learn_rate = 0.001
num_epoches = 2
# 多分类损失函数，使用默认值
criterion = nn.CrossEntropyLoss()  
# 梯度下降，求解模型最后一层参数
optimizer = optim.SGD(my_resnet50.fc.parameters(),lr=learn_rate,momentum=0.9)
# 判断使用CPU还是GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# 图片分批次送入内存（32张图片,batch_size），进行计算。
train_dataloader = DataLoader(dataset=train_dataset,batch_size=32,shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset)

In [0]:
# 训练阶段
my_resnet50.to(device)
my_resnet50.train()
for epoch in range(num_epoches):
    print(f"epoch: {epoch+1}")
    for idx,(img,label)in enumerate(train_dataloader):
        images = img.to(device)
        labels = label.to(device)
        output = my_resnet50(images)
        loss = criterion(output,labels)
        loss.backward() # 损失反向传播
        optimizer.step() # 更新梯度
        optimizer.zero_grad() # 梯度清零
        if idx%100==0:
            print(f"current loss = {loss.item()}")


In [0]:
# 超参数设置
learn_rate = 0.0001
num_epoches = 5
for epoch in range(num_epoches):
    print(f"epoch: {epoch+1}")
    for idx,(img,label)in enumerate(train_dataloader):
        images = img.to(device)
        labels = label.to(device)
        output = my_resnet50(images)
        loss = criterion(output,labels)
        loss.backward() # 损失反向传播
        optimizer.step() # 更新梯度
        optimizer.zero_grad() # 梯度清零
        if idx%100==0:
            print(f"current loss = {loss.item()}")

In [0]:

# 测试阶段
my_resnet50.to(device)
my_resnet50.eval() # 把训练好的模型的参数冻结
total,correct = 0 , 0
for img,label in test_dataloader:
    images = img.to(device)
    labels = label.to(device)
    #print("label: ",labels)
    output = my_resnet50(images)
    #print("output:", output.data.size)
    _,idx = torch.max(output.data,1) # 输出最大值的位置
    #print("idx: ", idx)
    total += labels.size(0) # 全部图片
    correct += (idx==labels).sum() # 正确的图片
    #print("correct_num: %f",correct)
print("correct_num: ",correct)
print("total_image_num: ",total)
print(f"accuracy:{100.*correct/total}")   